# Load Required Packages

In [1]:
import pandas as pd
import numpy as np

# Convert to parquet and load the data

In [2]:
data = pd.read_csv(r"E:\datasets\car\all_anonymized_2015_11_2017_03.csv", low_memory=False)
IDXData = data.reset_index()
IDXData["index"] = "AD-" + IDXData["index"].astype("str")

In [3]:
Data = data.copy()
Data.head(2)

,index,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,color_slug,stk_year,transmission,door_count,seat_count,fuel_type,date_created,date_last_seen,price_eur
0,AD-0,ford,galaxy,151000.0,2011.0,2000.0,103.0,None,None,NaN,man,5.0,7.0,diesel,2015-11-14 18:10:06.838319+00,2016-01-27 20:40:15.46361+00,10584.75
1,AD-1,skoda,octavia,143476.0,2012.0,2000.0,81.0,None,None,NaN,man,5.0,5.0,diesel,2015-11-14 18:10:06.853411+00,2016-01-27 20:40:15.46361+00,8882.31


# Initial Cleaning

We need a list of reasonable columns with non-null values:
maker, model, mileage, manufacture_year, engine_displacement, engine_power, body_type, stk_year, transmission, door_count, seat_count, fuel_type, price_eur

Remove all cars with price of 1295.34. This seems like a default value or something and it is the price for electric, lpg, and CNG vehicles. There are 673623 cars with this price.

We need at least 12 non null values in each row. This yields around 1,468,236 cars.
Nulls Out of 15
0        5188
1      332598
2     1130450
3      619227
4      522809
5      268050
6      225938
7      199000
8      126744
9       88750
10      29882
11       4276
Name: index, dtype: int64

Replace None with nan.

In [4]:
Data = Data[(Data["maker"].notnull()) & (Data["model"].notnull())]

In [5]:
MostImportantColumnsData = Data[
    [
        "index",
        "maker",
        "model",
        "mileage",
        "manufacture_year",
        "engine_displacement",
        "engine_power",
        "body_type",
        "stk_year",
        "transmission",
        "door_count",
        "seat_count",
        "fuel_type",
        "price_eur",
    ]
].copy()

In [6]:
ReasonAblePrice = MostImportantColumnsData[
    (MostImportantColumnsData["price_eur"] >= 500)
    & (MostImportantColumnsData["price_eur"] <= 50000)
    & (MostImportantColumnsData["price_eur"] != 1295.34)
].copy()
len(ReasonAblePrice)

1779127

In [7]:
ReasonAblePrice["Nulls"] = ReasonAblePrice.isnull().sum(axis=1)

print(len(ReasonAblePrice.columns))
print(len(ReasonAblePrice))

ReasonAblePrice.groupby("Nulls")["index"].count()

15
1779127


Nulls
0      5024
1    229749
2    879909
3    245652
4    170815
5     85705
6     70017
7     83572
8      7050
9      1634
Name: index, dtype: int64

In [8]:
LowNullData = ReasonAblePrice.fillna(value=np.nan)
LowNullData = LowNullData.dropna(thresh=6)

len(LowNullData)

1779127

In [9]:
LowNullData.head()

,index,maker,model,mileage,manufacture_year,engine_displacement,engine_power,body_type,stk_year,transmission,door_count,seat_count,fuel_type,price_eur,Nulls
0,AD-0,ford,galaxy,151000.0,2011.0,2000.0,103.0,NaN,NaN,man,5.0,7.0,diesel,10584.75,2
1,AD-1,skoda,octavia,143476.0,2012.0,2000.0,81.0,NaN,NaN,man,5.0,5.0,diesel,8882.31,2
3,AD-3,skoda,fabia,111970.0,2004.0,1200.0,47.0,NaN,NaN,man,5.0,5.0,gasoline,2960.77,2
4,AD-4,skoda,fabia,128886.0,2004.0,1200.0,47.0,NaN,NaN,man,5.0,5.0,gasoline,2738.71,2
5,AD-5,skoda,fabia,140932.0,2003.0,1200.0,40.0,NaN,NaN,man,5.0,5.0,gasoline,1628.42,2


# Define a cleaning function

In [10]:
def FillNa(Row: pd.Series, TargetColumn: str, ColumnsList: list, WithValues: dict):
    if pd.isna(Row[TargetColumn]):
        idx = tuple([Row[x] for x in ColumnsList])
        if idx in WithValues:
            return WithValues[idx]
        else:
            return np.nan
    else:
        return Row[TargetColumn]

# Clean stk_year

In [11]:
Cleaned_STK_Year = LowNullData.copy()
Cleaned_STK_Year["stk_year"] = Cleaned_STK_Year["stk_year"].apply(
    lambda x: x if x < 2021 else 2020 if x < 2024 else np.nan
)

# Clean body_type

In [12]:
Target = "body_type"
CleanUsing = ["maker", "model", "manufacture_year"]

In [13]:
Cleaned_Body_type = Cleaned_STK_Year.copy()
Cleaned_Body_type[Target] = Cleaned_Body_type[Target].apply(
    lambda x: x if x != "other" and not pd.isna(x) else np.nan
)

In [14]:
BT_Groups = (
    Cleaned_Body_type[Cleaned_Body_type[Target].notnull()]
    .groupby(CleanUsing)[Target]
    .agg(pd.Series.mode)
)

BT_Groups = BT_Groups.apply(lambda x: x if type(x) is str else x[0]).reset_index()
BT_Groups = BT_Groups.set_index(CleanUsing).to_dict()[Target]

In [15]:
Cleaned_Body_type[Target] = Cleaned_Body_type.apply(
    lambda x: FillNa(x, Target, CleanUsing, BT_Groups), axis=1
)

# Clean engine_displacement

Engine Displacement Minimum of 600 and max of 8500, else replace with nan

https://www.drivespark.com/four-wheelers/2016/five-smallest-displacement-engines-production-cars/articlecontent-pf50713-019057.html

https://www.caranddriver.com/features/g21784022/supersize-me-the-16-largest-displacement-engines-you-can-buy-today/?slide=16




In [16]:
Target = "engine_displacement"
CleanUsing = ["maker", "model", "manufacture_year", "transmission"]

In [17]:
CleanedEngineDisplacement = Cleaned_Body_type.copy()
CleanedEngineDisplacement[Target] = CleanedEngineDisplacement[Target].apply(
    lambda x: x if x >= 600 and x <= 8500 else np.nan
)

In [18]:
EDP_Groups = (
    CleanedEngineDisplacement[CleanedEngineDisplacement[Target].notnull()]
    .groupby(CleanUsing)[Target]
    .agg(pd.Series.median)
)

EDP_Groups = EDP_Groups.to_dict()

In [19]:
CleanedEngineDisplacement[Target] = CleanedEngineDisplacement.apply(
    lambda x: FillNa(x, Target, CleanUsing, EDP_Groups), axis=1
)

# Clean engine_power

https://en.wikipedia.org/wiki/List_of_production_cars_by_power_output

https://www.autotrader.com/oversteer/the-5-least-powerful-cars-in-2021-vs-2001

In [20]:
Target = "engine_power"
CleanUsing = ["maker", "model", "manufacture_year", "transmission"]

In [21]:
ClearnedEnginePower = CleanedEngineDisplacement.copy()
ClearnedEnginePower["engine_power"] = ClearnedEnginePower["engine_power"] * 1.34102

ClearnedEnginePower[Target] = ClearnedEnginePower[Target].apply(
    lambda x: x if x >= 65 and x <= 500 else np.nan
)

In [22]:
EPP_Groups = (
    ClearnedEnginePower[ClearnedEnginePower[Target].notnull()]
    .groupby(CleanUsing)[Target]
    .agg(pd.Series.median)
)

EPP_Groups = EPP_Groups.to_dict()

In [23]:
ClearnedEnginePower[Target] = ClearnedEnginePower.apply(
    lambda x: FillNa(x, Target, CleanUsing, EPP_Groups), axis=1
)

# Split the data based on different missing values

In [24]:
FinalCopy = ClearnedEnginePower.copy()
FinalCopy["Nulls"] = FinalCopy.isnull().sum(axis=1)

print(len(FinalCopy.columns))
print(len(FinalCopy))

FinalCopy.groupby("Nulls")["index"].count()

15
1779127


Nulls
0      13509
1     463633
2     552324
3     368870
4     158207
5      73050
6      78166
7      41841
8      24792
9       2994
10      1741
Name: index, dtype: int64

In [26]:
FinalCopy[FinalCopy["Nulls"] == 10].to_clipboard()

In [51]:
LearningCopy = FinalCopy.copy()

LearningCopy = LearningCopy.drop(columns=["index", "Nulls"])

StatusFrame = LearningCopy.notnull()

options = LearningCopy.notnull().drop_duplicates().reset_index(drop=True)

In [82]:
groups = list()

for i, o in options.iterrows():
    MatchingMask = StatusFrame.eq(o).all(axis=1)
    MatchingRows = LearningCopy[MatchingMask]
    tuple = (i, o, len(MatchingRows), MatchingRows)
    groups.append(tuple)

In [86]:
Occurences = [x[2] for x in groups]
options["Occurences"] = Occurences

In [88]:
options.to_clipboard()